# DuckDB.mojo examples

## Connect to a database

In [1]:
from duckdb import *
var con = DuckDB.connect(":memory:")
# var con = DuckDB.connect("train_services.duckdb")

## Read data from a remote Parquet file

In [2]:
_ = con.execute("""
CREATE TABLE train_services AS
FROM 's3://duckdb-blobs/train_services.parquet';
""")

## Run a query and extract the result

In [3]:
var result: MaterializedResult = con.execute(
    """
-- Get the top-3 busiest train stations
SELECT station_name, count(*) AS num_services
FROM train_services
GROUP BY ALL
ORDER BY num_services DESC
LIMIT 3;
"""
).fetch_all()

for col in result.columns():
    print(col[])

Column(0, station_name: varchar)
Column(1, num_services: bigint)


In [4]:
for row in range(len(result)):
    print(
        result.get(varchar, col=0, row=row).value(),
        " ",
        result.get(bigint, col=1, row=row).value(),
    )

Utrecht Centraal   7663
Amsterdam Centraal   7591
Zwolle   5013


## Use one of DuckDB's many integrations

In [5]:
_ = con.execute("COPY (select * from train_services limit 100) TO 'train_services.csv'")

## Working with lists

In [6]:
var list_result = con.execute("SELECT ['a', 'b', 'c']").fetch_all()
for col in list_result.columns():
    print(col[])

Column(0, main.list_value('a', 'b', 'c'): list(varchar))


In [7]:
# print helper
from collections import Optional
fn print_list[T: StringableCollectionElement](l: List[Optional[T]]):
    var ls = List[String]()
    for i in l:
        if i[]:
            ls.append(str(i[].value()))
        else:
            ls.append("NULL")
    print("[", (",").join(ls), "]", sep="")

In [8]:
var l: Optional[List[Optional[String]]] = list_result.get(list(varchar), col=0, row=0)
print_list(l.value())

[a,b,c]
